"This demonstrates that [...] we cannot arbitrarily
increase the number of features when the parameters of
class-conditional densities are estimated from a finite
number of training samples. The practical implication of
the curse of dimensionality is that a system designer should
try to select only a small number of salient features when
confronted with a limited training set."